# 国勢調査から年収の予測 「$50,000を超えるか否か」

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
sample = pd.read_csv("./sample_submit.csv",header=None)
print("Data Shapes")
print("Train:",train.shape, "Test:",test.shape, "Sample:",sample.shape)

Data Shapes
Train: (10200, 13) Test: (6800, 12) Sample: (6800, 2)


### データセットの調査

In [2]:
print(train.isnull().sum())
print()
print(test.isnull().sum())

index             0
age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
native-country    0
Y                 0
dtype: int64

id                0
age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
native-country    0
dtype: int64


In [3]:
train

,index,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,native-country,Y
0,0,22,Private,132618,Some-college,12,Never-married,Tech-support,Own-child,White,Female,United-States,0
1,1,22,Private,132655,Bachelors,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,Mexico,1
2,2,23,?,132674,HS-grad,8,Never-married,Other-service,Not-in-family,White,Female,United-States,0
3,3,36,Private,132642,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,United-States,1
4,4,49,Private,132646,Assoc-voc,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10195,10195,31,Private,132650,Bachelors,9,Divorced,Craft-repair,Other-relative,White,Female,Mexico,0
10196,10196,36,Private,132590,HS-grad,9,Married-civ-spouse,Exec-managerial,Unmarried,White,Male,Mexico,0
10197,10197,36,Local-gov,132498,Bachelors,13,Married-civ-spouse,?,Husband,Black,Male,United-States,0
10198,10198,17,Private,132614,Some-college,9,Never-married,?,Not-in-family,White,Female,United-States,0


### 文字列データを数値に変換

In [4]:
train['workclass_number'] = 0
train['education_number'] = 0
train['marital-status_number']= 0
train['occupation_number'] = 0
train['relationship_number'] = 0
train['race_number'] = 0
train['sex_number'] = 0
train['native-country_number'] = 0

test['workclass_number'] = 0
test['education_number'] = 0
test['marital-status_number']= 0
test['occupation_number'] = 0
test['relationship_number'] = 0
test['race_number'] = 0
test['sex_number'] = 0
test['native-country_number'] = 0

train.head()

,index,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,...,native-country,Y,workclass_number,education_number,marital-status_number,occupation_number,relationship_number,race_number,sex_number,native-country_number
0,0,22,Private,132618,Some-college,12,Never-married,Tech-support,Own-child,White,...,United-States,0,0,0,0,0,0,0,0,0
1,1,22,Private,132655,Bachelors,9,Married-civ-spouse,Farming-fishing,Husband,White,...,Mexico,1,0,0,0,0,0,0,0,0
2,2,23,?,132674,HS-grad,8,Never-married,Other-service,Not-in-family,White,...,United-States,0,0,0,0,0,0,0,0,0
3,3,36,Private,132642,Bachelors,13,Married-civ-spouse,Sales,Husband,White,...,United-States,1,0,0,0,0,0,0,0,0
4,4,49,Private,132646,Assoc-voc,13,Married-civ-spouse,Exec-managerial,Husband,White,...,United-States,1,0,0,0,0,0,0,0,0


In [5]:
u = train["workclass"].unique()
print(u)
print(type(u))

['Private' '?' 'Local-gov' 'Self-emp-inc' 'Federal-gov' 'Self-emp-not-inc'
 'State-gov']
<class 'numpy.ndarray'>


In [6]:
train.loc[train["workclass"] == "Private","workclass_number"] = 1
train.loc[train["workclass"] == "Local-gov","workclass_number"] = 2
train.loc[train["workclass"] == "Self-emp-inc","workclass_number"] = 3
train.loc[train["workclass"] == "Federal-gov","workclass_number"] = 4
train.loc[train["workclass"] == "Self-emp-not-inc","workclass_number"] = 5
train.loc[train["workclass"] == "State-gov","workclass_number"] = 6

test.loc[train["workclass"] == "Private","workclass_number"] = 1
test.loc[train["workclass"] == "Local-gov","workclass_number"] = 2
test.loc[train["workclass"] == "Self-emp-inc","workclass_number"] = 3
test.loc[train["workclass"] == "Federal-gov","workclass_number"] = 4
test.loc[train["workclass"] == "Self-emp-not-inc","workclass_number"] = 5
test.loc[train["workclass"] == "State-gov","workclass_number"] = 6


In [7]:
u = train["education"].unique()
print(u)
print(type(u))

['Some-college' 'Bachelors' 'HS-grad' 'Assoc-voc' '12th' '10th' 'Masters'
 '5th-6th' 'Assoc-acdm' 'Prof-school' '7th-8th' '11th' '9th' '1st-4th']
<class 'numpy.ndarray'>


In [8]:
train.loc[train["education"] == "Some-college","education_number"] = 1
train.loc[train["education"] == "Bachelors","education_number"] = 2
train.loc[train["education"] == "HS-grad","education_number"] = 3
train.loc[train["education"] == "Assoc-voc","education_number"] = 4
train.loc[train["education"] == "12th","education_number"] = 5
train.loc[train["education"] == "10th","education_number"] = 6
train.loc[train["education"] == "Masters","education_number"] = 7
train.loc[train["education"] == "5th-6th","education_number"] = 8
train.loc[train["education"] == "Assoc-acdm","education_number"] = 9
train.loc[train["education"] == "Prof-school","education_number"] = 10
train.loc[train["education"] == "7th-8th","education_number"] = 11
train.loc[train["education"] == "11th","education_number"] = 12
train.loc[train["education"] == "9th","education_number"] = 13
train.loc[train["education"] == "1st-4th","education_number"] = 14

test.loc[train["education"] == "Some-college","education_number"] = 1
test.loc[train["education"] == "Bachelors","education_number"] = 2
test.loc[train["education"] == "HS-grad","education_number"] = 3
test.loc[train["education"] == "Assoc-voc","education_number"] = 4
test.loc[train["education"] == "12th","education_number"] = 5
test.loc[train["education"] == "10th","education_number"] = 6
test.loc[train["education"] == "Masters","education_number"] = 7
test.loc[train["education"] == "5th-6th","education_number"] = 8
test.loc[train["education"] == "Assoc-acdm","education_number"] = 9
test.loc[train["education"] == "Prof-school","education_number"] = 10
test.loc[train["education"] == "7th-8th","education_number"] = 11
test.loc[train["education"] == "11th","education_number"] = 12
test.loc[train["education"] == "9th","education_number"] = 13
test.loc[train["education"] == "1st-4th","education_number"] = 14


In [9]:
u = train["marital-status"].unique()
print(u)
print(type(u))

['Never-married' 'Married-civ-spouse' 'Divorced' 'Separated' 'Widowed']
<class 'numpy.ndarray'>


In [10]:
train.loc[train["marital-status"] == "Never-married","marital-status_number"] = 1
train.loc[train["marital-status"] == "Married-civ-spouse","marital-status_number"] = 2
train.loc[train["marital-status"] == "Divorced","marital-status_number"] = 3
train.loc[train["marital-status"] == "Separated","marital-status_number"] = 4
train.loc[train["marital-status"] == "Widowed","marital-status_number"] = 5

test.loc[train["marital-status"] == "Never-married","marital-status_number"] = 1
test.loc[train["marital-status"] == "Married-civ-spouse","marital-status_number"] = 2
test.loc[train["marital-status"] == "Divorced","marital-status_number"] = 3
test.loc[train["marital-status"] == "Separated","marital-status_number"] = 4
test.loc[train["marital-status"] == "Widowed","marital-status_number"] = 5


In [11]:
u = train["occupation"].unique()
print(u)
print(type(u))

['Tech-support' 'Farming-fishing' 'Other-service' 'Sales'
 'Exec-managerial' 'Protective-serv' 'Adm-clerical' 'Prof-specialty'
 'Handlers-cleaners' 'Craft-repair' '?' 'Machine-op-inspct'
 'Transport-moving']
<class 'numpy.ndarray'>


In [12]:
train.loc[train["occupation"] == "Tech-support","occupation_number"] = 1
train.loc[train["occupation"] == "Farming-fishing","occupation_number"] = 2
train.loc[train["occupation"] == "Other-service","occupation_number"] = 3
train.loc[train["occupation"] == "Sales","occupation_number"] = 4
train.loc[train["occupation"] == "Exec-managerial","occupation_number"] = 5
train.loc[train["occupation"] == "Protective-serv","occupation_number"] = 6
train.loc[train["occupation"] == "Adm-clerical","occupation_number"] = 7
train.loc[train["occupation"] == "Prof-specialty","occupation_number"] = 8
train.loc[train["occupation"] == "Handlers-cleaners","occupation_number"] = 9
train.loc[train["occupation"] == "Craft-repair","occupation_number"] = 10
train.loc[train["occupation"] == "Machine-op-inspct","occupation_number"] = 11
train.loc[train["occupation"] == "Transport-moving","occupation_number"] = 12

test.loc[train["occupation"] == "Tech-support","occupation_number"] = 1
test.loc[train["occupation"] == "Farming-fishing","occupation_number"] = 2
test.loc[train["occupation"] == "Other-service","occupation_number"] = 3
test.loc[train["occupation"] == "Salese","occupation_number"] = 4
test.loc[train["occupation"] == "Exec-managerial","occupation_number"] = 5
test.loc[train["occupation"] == "Protective-serv","occupation_number"] = 6
test.loc[train["occupation"] == "Adm-clerical","occupation_number"] = 7
test.loc[train["occupation"] == "Prof-specialty","occupation_number"] = 8
test.loc[train["occupation"] == "Handlers-cleaners","occupation_number"] = 9
test.loc[train["occupation"] == "Craft-repair","occupation_number"] = 10
test.loc[train["occupation"] == "Machine-op-inspct","occupation_number"] = 11
test.loc[train["occupation"] == "Transport-moving","occupation_number"] = 12


In [13]:
u = train["relationship"].unique()
print(u)
print(type(u))

['Own-child' 'Husband' 'Not-in-family' 'Unmarried' 'Wife' 'Other-relative']
<class 'numpy.ndarray'>


In [14]:
train.loc[train["relationship"] == "Own-child","relationship_number"] = 1
train.loc[train["relationship"] == "Husband","relationship_number"] = 2
train.loc[train["relationship"] == "Not-in-family","relationship_number"] = 3
train.loc[train["relationship"] == "Unmarried","relationship_number"] = 4
train.loc[train["relationship"] == "Wife","relationship_number"] = 5
train.loc[train["relationship"] == "Other-relative","relationship_number"] = 6

test.loc[train["relationship"] == "Own-child","relationship_number"] = 1
test.loc[train["relationship"] == "Husband","relationship_number"] = 2
test.loc[train["relationship"] == "Not-in-family","relationship_number"] = 3
test.loc[train["relationship"] == "Unmarried","relationship_number"] = 4
test.loc[train["relationship"] == "Wife","relationship_number"] = 5
test.loc[train["relationship"] == "Other-relative","relationship_number"] = 6


In [15]:
u = train["race"].unique()
print(u)
print(type(u))

['White' 'Asian-Pac-Islander' 'Black']
<class 'numpy.ndarray'>


In [16]:
train.loc[train["race"] == "White","race_number"] = 1
train.loc[train["race"] == "Asian-Pac-Islander","race_number"] = 2
train.loc[train["race"] == "Black","race_number"] = 3

test.loc[train["race"] == "White","race_number"] = 1
test.loc[train["race"] == "Asian-Pac-Islander","race_number"] = 2
test.loc[train["race"] == "Black","race_number"] = 3


In [17]:
u = train["sex"].unique()
print(u)
print(type(u))

['Female' 'Male']
<class 'numpy.ndarray'>


In [18]:
train.loc[train["sex"] == "Female","sex_number"] = 1
train.loc[train["sex"] == "Male","sex_number"] = 2

test.loc[train["sex"] == "Female","sex_number"] = 1
test.loc[train["sex"] == "Male","sex_number"] = 2


In [19]:
u = train["native-country"].unique()
print(u)
print(type(u))

['United-States' 'Mexico' 'Philippines']
<class 'numpy.ndarray'>


In [20]:
train.loc[train["native-country"] == "United-States","native-country_number"] = 1
train.loc[train["native-country"] == "Mexico","native-country_number"] = 2
train.loc[train["native-country"] == "Philippines","native-country_number"] = 3

test.loc[train["native-country"] == "United-States","native-country_number"] = 1
test.loc[train["native-country"] == "Mexico","native-country_number"] = 2
test.loc[train["native-country"] == "Philippines","native-country_number"] = 3

train

,index,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,...,native-country,Y,workclass_number,education_number,marital-status_number,occupation_number,relationship_number,race_number,sex_number,native-country_number
0,0,22,Private,132618,Some-college,12,Never-married,Tech-support,Own-child,White,...,United-States,0,1,1,1,1,1,1,1,1
1,1,22,Private,132655,Bachelors,9,Married-civ-spouse,Farming-fishing,Husband,White,...,Mexico,1,1,2,2,2,2,1,2,2
2,2,23,?,132674,HS-grad,8,Never-married,Other-service,Not-in-family,White,...,United-States,0,0,3,1,3,3,1,1,1
3,3,36,Private,132642,Bachelors,13,Married-civ-spouse,Sales,Husband,White,...,United-States,1,1,2,2,4,2,1,2,1
4,4,49,Private,132646,Assoc-voc,13,Married-civ-spouse,Exec-managerial,Husband,White,...,United-States,1,1,4,2,5,2,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10195,10195,31,Private,132650,Bachelors,9,Divorced,Craft-repair,Other-relative,White,...,Mexico,0,1,2,3,10,6,1,1,2
10196,10196,36,Private,132590,HS-grad,9,Married-civ-spouse,Exec-managerial,Unmarried,White,...,Mexico,0,1,3,2,5,4,1,2,2
10197,10197,36,Local-gov,132498,Bachelors,13,Married-civ-spouse,?,Husband,Black,...,United-States,0,2,2,2,0,2,3,2,1
10198,10198,17,Private,132614,Some-college,9,Never-married,?,Not-in-family,White,...,United-States,0,1,1,1,0,3,1,1,1


### 不要列データの削除

In [21]:
train = train.drop(columns=["workclass","education","marital-status","occupation","relationship","race","sex","native-country"])
test = test.drop(columns=["workclass","education","marital-status","occupation","relationship","race","sex","native-country"])
train.head()

,index,age,fnlwgt,education-num,Y,workclass_number,education_number,marital-status_number,occupation_number,relationship_number,race_number,sex_number,native-country_number
0,0,22,132618,12,0,1,1,1,1,1,1,1,1
1,1,22,132655,9,1,1,2,2,2,2,1,2,2
2,2,23,132674,8,0,0,3,1,3,3,1,1,1
3,3,36,132642,13,1,1,2,2,4,2,1,2,1
4,4,49,132646,13,1,1,4,2,5,2,1,2,1


In [22]:
test.head()

,id,age,fnlwgt,education-num,workclass_number,education_number,marital-status_number,occupation_number,relationship_number,race_number,sex_number,native-country_number
0,10200,30,132646,12,1,1,1,1,1,1,1,1
1,10201,30,132626,4,1,2,2,2,2,1,2,2
2,10202,36,132505,8,0,3,1,3,3,1,1,1
3,10203,23,132610,9,1,2,2,0,2,1,2,1
4,10204,36,132468,9,1,4,2,5,2,1,2,1


### 目的変数の抽出

In [23]:
y = train["Y"]
print(y)

0        0
1        1
2        0
3        1
4        1
        ..
10195    0
10196    0
10197    0
10198    0
10199    0
Name: Y, Length: 10200, dtype: int64


In [24]:
train = train.drop(columns=["Y"])
train.head()

,index,age,fnlwgt,education-num,workclass_number,education_number,marital-status_number,occupation_number,relationship_number,race_number,sex_number,native-country_number
0,0,22,132618,12,1,1,1,1,1,1,1,1
1,1,22,132655,9,1,2,2,2,2,1,2,2
2,2,23,132674,8,0,3,1,3,3,1,1,1
3,3,36,132642,13,1,2,2,4,2,1,2,1
4,4,49,132646,13,1,4,2,5,2,1,2,1


In [25]:
train_data = train.values
train_labels = y
test_data = test.values


### 正規化

In [26]:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)
train_data = (train_data - mean) / std
test_data = (test_data - mean) / std


### モデルの構築

In [27]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='mse', 
              metrics=['mae'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Using TensorFlow backend.


In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                832       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 5,057
Trainable params: 5,057
Non-trainable params: 0
_________________________________________________________________


### 学習

In [29]:
history = model.fit(train_data, 
                    train_labels,
                    batch_size=128,
                    epochs=200,
                    verbose=1)


Epoch 1/200
10200/10200 [==============================] - 1s 56us/step - loss: 0.1337 - mae: 0.2954
Epoch 2/200
10200/10200 [==============================] - 0s 30us/step - loss: 0.1054 - mae: 0.2192
Epoch 3/200
10200/10200 [==============================] - 0s 25us/step - loss: 0.1001 - mae: 0.2092
Epoch 4/200
10200/10200 [==============================] - 0s 29us/step - loss: 0.0957 - mae: 0.1988
Epoch 5/200
10200/10200 [==============================] - 0s 28us/step - loss: 0.0927 - mae: 0.1892
Epoch 6/200
10200/10200 [==============================] - 0s 23us/step - loss: 0.0902 - mae: 0.1836
Epoch 7/200
10200/10200 [==============================] - 0s 28us/step - loss: 0.0888 - mae: 0.1781
Epoch 8/200
10200/10200 [==============================] - 0s 28us/step - loss: 0.0882 - mae: 0.1747
Epoch 9/200
10200/10200 [==============================] - 0s 28us/step - loss: 0.0872 - mae: 0.1718
Epoch 10/200
10200/10200 [==============================] - 0s 27us/step - loss: 0.0865 - m

10200/10200 [==============================] - 0s 12us/step - loss: 0.0696 - mae: 0.1348
Epoch 82/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0697 - mae: 0.1348
Epoch 83/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0697 - mae: 0.1341
Epoch 84/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0691 - mae: 0.1340
Epoch 85/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0689 - mae: 0.1334
Epoch 86/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0690 - mae: 0.1335
Epoch 87/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0689 - mae: 0.1335
Epoch 88/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0690 - mae: 0.1327
Epoch 89/200
10200/10200 [==============================] - 0s 11us/step - loss: 0.0684 - mae: 0.1324
Epoch 90/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0678 - mae: 

10200/10200 [==============================] - 0s 12us/step - loss: 0.0591 - mae: 0.1144
Epoch 161/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0593 - mae: 0.1146
Epoch 162/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0587 - mae: 0.1131
Epoch 163/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0589 - mae: 0.1135
Epoch 164/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0583 - mae: 0.1134
Epoch 165/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0583 - mae: 0.1128
Epoch 166/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0582 - mae: 0.1131
Epoch 167/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0578 - mae: 0.1126
Epoch 168/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.0578 - mae: 0.1121
Epoch 169/200
10200/10200 [==============================] - 0s 12us/step - loss: 0.058

In [30]:
score = model.evaluate(train_data, train_labels, verbose=2)
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.052686964274548435
Test accuracy: 0.10356473177671432


### 予測

In [31]:
test_predictions = model.predict(test_data)

In [32]:
print(test_predictions.shape)

(6800, 1)


In [39]:
l =np.zeros(len(test_predictions),dtype=np.int64)
idx = 0
for t in test_predictions:
    if t > 0.4:
        l[idx] = 1
    else:
        l[idx] = 0
    idx += 1

In [40]:
sample[1] =l
sample

,0,1
0,10200,0
1,10201,1
2,10202,0
3,10203,0
4,10204,0
...,...,...
6795,16995,0
6796,16996,0
6797,16997,0
6798,16998,0


### ファイル出力

In [36]:
import csv
f = open('out.csv','w',newline='')
writer = csv.writer(f)
writer.writerows(sample.values)
f.close()

In [206]:
model.save('beginner_model')